In [1]:
import numpy as np #For array managment
import math
import random
import sys
sys.path.append('..')
from Simulation import simulation as sim

pygame 2.1.0 (SDL 2.0.16, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def DKL(l1,l2):
    ###Kullback-Leibler divergence between two arrays, they have to be pmfs (ie histograms that sum to 1)
    return(np.sum([l1[i]*math.log(l1[i]/l2[i]) for i in range(len(l1)) if l1[i]!=0]))

In [3]:
S = 3 #For each move, we have four contributors

#declared by the user range(3-10) from 3x3 to 10x10
grid_dimension = 5
fields_number = grid_dimension*grid_dimension
grid = []
state_memory = []

In [4]:
def findWall(x):
    ###Check if the agent is near a wall on square x
    # Returns a list of ints, one for each wall (in Python 1<=>True, 0<=>False)
    res = [0, 0, 0, 0]
    if x % grid_dimension == 0:  # If the agent is near the left-side wall, update the list
        res[0] = 1
    if x % grid_dimension == (grid_dimension - 1):  # If the agent is near the right-side wall, update the list
        res[2] = 1
    if x // grid_dimension == 0:  # If the agent is near the bottom-side wall, update the list
        res[3] = 1
    if x // grid_dimension == (grid_dimension - 1):  # If the agent is near the top-side wall, update the list
        res[1] = 1

    return (res)  # Return the list

In [5]:
def target_direction():
    # arrange the list in the correct order
    startindex = grid_dimension
    stopindex = 0
    step = -1
    field_number = 0
    cnt = 0
    for i in range(startindex, stopindex, step):
        for j in range(0, grid_dimension, 1):
            field_number = (grid_dimension * i) - grid_dimension + j
            grid.insert(cnt, field_number)
            cnt += 1

    ## Fields for whose GO right determined on the basis of the diagonal
    squaresGoRight = []
    rows = grid_dimension;  # const value
    cols = grid_dimension;
    counter = 0;

    for j in range(fields_number):
        if (counter == rows):  # in 1st row take all, in next take 1 less
            counter = 0
            cols = cols - 1  # decrise
        if (counter < cols):
            squaresGoRight.append(grid[j])
        counter = counter + 1

    ## Fields for whose GO up, rest of the grid
    squaresGoUp = [squareNumber for squareNumber in grid if squareNumber not in squaresGoRight]
    return squaresGoRight, squaresGoUp

In [6]:
def return_target(x):
    ###Return the pmf corresponding to the target for square x (the behavior is picked depending on the square)

    squaresGoRight, squaresGoUp = target_direction()

    if x in squaresGoRight:  # For these squares, we want to go right
        return ([0.01, 0.01, 0.01, 0.96, 0.01])  # Return the pmf corresponding to moving right

    if x in squaresGoUp:  # For these squares, we want to go up
        return ([0.01, 0.01, 0.96, 0.01, 0.01])  # Return the pmf corresponding to moving top

    return ([0.96, 0.01, 0.01, 0.01, 0.01])  # Else, we want to stay on position

In [7]:
def rand_obstacle_location():
    # rand obstacle location
    # get diffent amount of obstacle depended on grid size, for 3x3 1 obstacle, 4x4 2obstacle etc.

    if (grid_dimension == 3):
        number_of_obtacles = 1;
    else:
        number_of_obtacles = (grid_dimension - 2)

    field_number_with_obtacle = []

    for i in range(number_of_obtacles):
        result = random.randrange(1, (fields_number - 1))  # without 1st and last square (for 5x5) w/o 0 and 24
        field_number_with_obtacle.append(result)

    print(f"Fields with obstacles: {field_number_with_obtacle}")
    return field_number_with_obtacle

In [8]:
def return_reward(x):

    ###Return the reward function (voded as a list of its values) for square x
    # Square zero (the starting point) has a reward of -1, and square 4 (the middle) has a reward of -100
    # Again, this function picks the reward based on the square where the agent is
    if (x + grid_dimension) in field_number_with_obtacle and not (x + 1) in field_number_with_obtacle:
        return ([-1, -1, -100, 0, -1])  # obstacle ABOVE agent
    if (x + 1) in field_number_with_obtacle and not (x + grid_dimension) in field_number_with_obtacle:
        return ([-1, -1, 0, -100, -1])  # obstacle RIGHT side agent
    if (x - 1) in field_number_with_obtacle:
        return ([0, -100, 0, 0, 0])  # obstacle LEFT agent
    if (x - grid_dimension) in field_number_with_obtacle:
        return ([0, 0, 0, 0, -100])  # obstacle BELOW agent
    if x in field_number_with_obtacle:
        return ([-100, 0, 0, 0, 0])  # ARRR! agent in the obstacle
    if (x + grid_dimension) in field_number_with_obtacle and (x + 1) in field_number_with_obtacle:
        return [0, 0, -1000, -1000, 0]  # obstacle ABOVE and RIGHT side of agent
    #if x == 0:
      #  return ([-1, 0, 0, 0, 0])  # starting point
    return ([0, 0, 0, 0, 0])  # else

In [9]:
def return_sources(x):
    ###Return the behavior of the sources for square x
    source_left = [0.01, 0.96, 0.01, 0.01, 0.01]
    source_top = [0.01, 0.01, 0.96, 0.01, 0.01]
    source_right = [0.01, 0.01, 0.01, 0.96, 0.01]
    source_bot = [0.01, 0.01, 0.01, 0.01, 0.96]
    # source_stay = [0.96, 0.01, 0.01, 0.01, 0.01]

    if findWall(x)[0]:  # If we are near a wall (or more), the corresponding movement is replaced by "stay in place"
        source_left = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[1]:
        source_top = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[2]:
        source_right = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[3]:
        source_bot = [0.96, 0.01, 0.01, 0.01, 0.01]

    return (np.array([source_left, source_top, source_right, source_bot]))

In [10]:
def new_state(position, action):
    if action == 0:
        return(position)
    if action == 1:
        return(position - 1)
    if action == 2:
        return(position + grid_dimension)
    if action == 3:
        return(position + 1)
    if action == 4:
        return(position - grid_dimension)

In [11]:
def control_algorithm(actual_position):
    #     free_grids = findWall(actual_position) #check if there's no walls
    #     available states = [i for i, x in enumerate(free_grids) if x == 0]
    weights = np.zeros(S)

    target = return_target(actual_position)
    reward = return_reward(actual_position)
    sources = return_sources(actual_position)

    for i in range(S):
        result_DKL = DKL(sources[i], target) - np.dot(sources[i], reward)  # Calculate the weight and store it
        weights[i] = result_DKL

    j = np.argmin(weights)  # Get the minimum weight
    return (return_sources(actual_position)[j])  # Return the corresponding policy

In [12]:
def loop(actual_position):
    ###
    #Simulation loop
    pmf = control_algorithm(actual_position)
    action = np.argmax(pmf) #We use a maximum sampling
    x = new_state(actual_position, action) #Get the corresponding new state
    print('PMF: ' + str(pmf))
    print('New state: ' + str(x))
    return(x)

In [ ]:
#main
x = 0
state_memory.append(x)
field_number_with_obtacle = rand_obstacle_location()
# field_number_with_obtacle = [4]
for i in range(fields_number-1):
    if(x != fields_number-1):
        x = loop(x)
        state_memory.append(x)

sim(grid_dimension, field_number_with_obtacle, state_memory)

Fields with obstacles: [1, 6, 22]
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 5
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 10
PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 11
PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 12
PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 13
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 18
PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 19
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 24
